In [ ]:
import polars as pl

pl.Config().set_tbl_rows(99)


class Meal:
    def __init__(self, meal: str, day: int, ingredients: list[list[str | float]]):
        self.day = day
        self.meal = meal
        self.ing_schema = {"ing": str, "unit": str, "val": float}
        self.schema = {"day": int, "name": str, **self.ing_schema}
        self.df = pl.DataFrame(schema=self.schema)

        if ingredients:
            for ingredient in ingredients:
                self.add_ingredients(*ingredient)

    def add_ingredients(self, name: str, unit: str, quantity: float):
        ing_df = pl.from_records(
            [[self.day, self.meal, name, unit, quantity]],
            schema=self.schema,
            orient="row",
        )

        self.df = pl.concat([self.df, ing_df])

    def _repr_html_(self):
        return display(self.df)


def construct_meal_tbl(
    day_num: int, meal_name: str, ingredients: dict[str, float]
) -> pl.DataFrame:
    df = pl.DataFrame()

    df = df.with_columns(pl.lit(day_num).alias("day"), pl.lit(meal_name).alias("meal"))

    ing_df = pl.DataFrame(
        [
            pl.Series(name="ing", values=ingredients.keys()),
            pl.Series(name="quant", values=ingredients.values()),
        ],
    )

    df = df.join(
        ing_df,
        how="cross",
    )

    return df


In [ ]:
burger_ings = [
    ["buns", "patties", 6],
    ["sliced_cheese", "slices", 8],
    ["tomato", "whole", 1],
    ["mayo", "ml", 120],
    ["patties", "whole", 4],
]

meatloaf_ings = [
    ["impossible mince", "packet", 2],
    ["carrots", "whole", 2],
    ["peas", "cup", 1],
    ["eggs", "whole", 2],
    ["brown mushrooms", "whole", 4],
    ["parsley", "sprig", 1],
    ["ketchup", "cup", 0.5],
    ["breadcrumbs", "cup", 2],
]

pastastew_ings = [
    ["lentils", "can", 2],
    ["passata", "bottle", 1],
    ["carrots", "whole", 3],
    ["onions", "whole", 2],
]

frittata_ings = [
    ["eggs", "whole", 6],
    ["carrots", "whole", 2],
    ["leek", "whole", 0.333],
    ["potato", "whole", 3],
    ["peas", "cup", 1],
]
# meal_1 = construct_meal_tbl(
#     day_num=1,
#     meal_name="burgers",
#     ingredients={},
# )
# meal_2 = construct_meal_tbl(
#     day_num=2,
#     meal_name="pasta_stew",
#     ingredients={"lentils (cans)": 2, "passata": 1, "carrots": 3, "onions": 2},
# )

meal_1 = Meal(meal="burgers", day=1, ingredients=burger_ings)
meal_2 = Meal(meal="pasta stew", day=2, ingredients=pastastew_ings)
meal_3 = Meal(meal="frittata", day=3, ingredients=frittata_ings)
meal_4 = Meal(meal="meatloaf", day=4, ingredients=meatloaf_ings)
# meal3 = Meal(meal="meatloaf", day=1)
# for row in meatloaf_ings:
#     meal3.add_ingredients(*row)

# meals = [
#     meal_1,
#     meal_2,
#     #  meal_3
# ]
# meals = pl.concat(meals, how="vertical")
# meals
meals = [meal_1, meal_2, meal_3, meal_4]
tot_ings = pl.concat([meal.df for meal in meals])
tot_ings


In [ ]:
tot_ings.select("ing", "unit", "val").group_by(["ing", "unit"]).sum()
